# Tucil 2 **Strategi Algoritma**
Nama : Patrick Amadeus Irawan \
NIM : 13520109 \
Kelas : K-01

In [ ]:
import pandas as pd  
import matplotlib.pyplot as plt  
import scipy

from sklearn import datasets

In [ ]:
data = datasets.load_iris()

In [ ]:
df = pd.DataFrame(data.data, columns=data.feature_names)
df['Target'] = pd.DataFrame(data.target)
df

Enumerate Target Label from **Dataset**

In [ ]:
n_target = len(df.Target.value_counts())
d_target = {}
for i in range(n_target):
    d_target[i] = (df.Target.value_counts().index[i])

Select X, Y to be Projected with **Hull Convex**

In [ ]:
idx = 0
msg = ""
for i in range(len(df.columns)):
    msg += (str(i+1)+". "+str(df.columns[i]) + "\n\n")
idX,idY = input(msg + "Pick 2 columns to plot [ex :1 2] : ").split()
x,y = df.columns[int(idX)-1],df.columns[int(idY)-1]

In [ ]:
d_points = {}

for i in d_target:
    d_points[i] = [(df[x].values[j] , df[y].values[j]) for j in range(len(df[x])) if df.Target.values[j] == i]


#### **QuickSort** Implementation for points

In [ ]:
def quick_sort(points):
    if len(points) <= 1:
        return points
    else:
        pivot = points[0]
        less = [point for point in points[1:] if (point[0] < pivot[0]) or (point[0] == pivot[0] and point[1] < pivot[1])]
        greater = [point for point in points[1:] if (point[0] > pivot[0]) or (point[0] == pivot[0] and point[1] >= pivot[1])]
        return quick_sort(less) + [pivot] + quick_sort(greater)

In [ ]:
for key in d_points:
    d_points[key] = quick_sort(d_points[key])

In [ ]:
import math
def dist_pt_line(A,B,C,x,y):
    return abs(A*x + B*y + C)/(math.sqrt(A**2 + B**2))

#### Divide and Conquer **Convex Hull** algorithm

In [ ]:
# convex hull algorithm from array of points
def convex_hull(points):
    points = quick_sort(points)
    p_min , p_max = points[0] , points[-1]
    
    # Coefficent for linear line equation
    A,B,C = (p_max[1] - p_min[1]) , (p_min[0] - p_max[0]) , (p_min[0]*(p_min[1] - p_max[1]) + p_min[1]*(p_max[0] - p_min[0]))
    upper,lower = [],[]
    for point in points:
        if (A*point[0] + B*point[1] + C) < 0:
            upper.append(point)
        elif (A*point[0] + B*point[1] + C) > 0:
            lower.append(point)
    return [p_min] + convex_hull_upper(upper,p_min,p_max) + [p_max]  +convex_hull_lower(lower,p_min,p_max)

def convex_hull_upper(points,p1,p2):
    if len(points) == 0:
        return []
    if len(points) == 1:
        return [points[0]]
    # Coefficent for linear line equation
    A,B,C = (p2[1] - p1[1]) , (p1[0] - p2[0]) , (p1[0]*(p1[1] - p2[1]) + p1[1]*(p2[0] - p1[0]))
    distance = 0
    p_max = None

    #find the maximum distance point
    for point in points:
        if dist_pt_line(A,B,C,point[0],point[1]) > distance:
            distance = dist_pt_line(A,B,C,point[0],point[1])
            p_max = point
    
    #prepare for leftside line
    A,B,C = (p_max[1] - p1[1]) , (p1[0] - p_max[0]) , (p1[0]*(p1[1] - p_max[1]) + p1[1]*(p_max[0] - p1[0]))
    upper1 = []
    for point in points:
        if (A*point[0] + B*point[1] + C) < 0:
            upper1.append(point)

    #prepare for rightside line
    A,B,C = (p2[1] - p_max[1]) , (p_max[0] - p2[0]) , (p_max[0]*(p_max[1] - p2[1]) + p_max[1]*(p2[0] - p_max[0]))
    upper2 = []
    for point in points:
        if (A*point[0] + B*point[1] + C) < 0:
            upper2.append(point)
    
    return convex_hull_upper(upper1 , p1 , p_max) + [p_max] + convex_hull_upper(upper2 , p_max , p2)

def convex_hull_lower(points,p1,p2):
    if len(points) == 0:
        return []
    if len(points) == 1:
        return [points[0]]
    # Coefficent for linear line equation
    A,B,C = (p2[1] - p1[1]) , (p1[0] - p2[0]) , (p1[0]*(p1[1] - p2[1]) + p1[1]*(p2[0] - p1[0]))
    distance = 0
    p_max = None

    #find the maximum distance point
    for point in points:
        if dist_pt_line(A,B,C,point[0],point[1]) > distance:
            distance = dist_pt_line(A,B,C,point[0],point[1])
            p_max = point
    
    #prepare for leftside line
    A,B,C = (p_max[1] - p1[1]) , (p1[0] - p_max[0]) , (p1[0]*(p1[1] - p_max[1]) + p1[1]*(p_max[0] - p1[0]))
    lower1 = []
    for point in points:
        if (A*point[0] + B*point[1] + C) > 0:
            lower1.append(point)

    #prepare for rightside line
    A,B,C = (p2[1] - p_max[1]) , (p_max[0] - p2[0]) , (p_max[0]*(p_max[1] - p2[1]) + p_max[1]*(p2[0] - p_max[0]))
    lower2 = []
    for point in points:
        if (A*point[0] + B*point[1] + C) > 0:
            lower2.append(point)
    
    return convex_hull_lower(lower2 , p_max , p2) + [p_max] + convex_hull_lower(lower1 , p1 , p_max)


#### **Hull Points** Gathering

In [ ]:
hull_points = {}

for i in range(n_target):
    d = convex_hull(d_points[i])
    hull_points[i] = convex_hull(d_points[i])


### Final Scatter Plot with **Convex Hull** 

In [ ]:
color_pallete = ["blue","green","red","purple","magenta","yellow","black","white"] # for plotting

f = plt.figure()
f.set_figwidth(7.5)
f.set_figheight(5)

for i in d_points:
    x, y = [pts[0] for pts in d_points[i]],[pts[1] for pts in d_points[i]]
    plt.scatter(x,y,c = color_pallete[i], label = data.target_names[i] ,s=20)

for i in hull_points:
    size = len(hull_points[i])
    for j in range(len(hull_points[i])):
        plt.plot([hull_points[i][j % size][0], hull_points[i][(j + 1)% size][0]], [hull_points[i][j % size][1], hull_points[i][(j + 1)% size][1]] , c = color_pallete[i], linewidth = 0.75)

plt.xlabel(df.columns[int(idX) - 1])
plt.ylabel(df.columns[int(idY) - 1])
plt.legend()

plt.show()

In [ ]:
# x = [i[0] for i in d_points[0]]
# y = [i[1] for i in d_points[0]]
# plt.scatter(x[1:-1],y[1:-1],c = "red")
# plt.scatter([x[0] , x[-1]],[y[0],y[-1]],c = "green")
# plt.plot([x[0] , x[-1]],[y[0],y[-1]],c = "green")
# plt.scatter([5.2,4.5],[4.1,2.3],c= "blue")


# plt.plot([x[0],5.2] ,[y[0] ,4.1] ,c= "blue")
# plt.plot([x[0],4.5] ,[y[0] ,2.3] ,c= "blue")
# plt.plot([x[-1],4.5] ,[y[-1] ,2.3] ,c= "blue")
# plt.plot([x[-1],5.2] ,[y[-1] ,4.1] ,c= "blue")

# plt.plot([x[0],4.6],[y[0],3.6],c="black")

# d = convex_hull(d_points[0])
# x,y = [point[0] for point in d],[point[1] for point in d]
# plt.scatter(x,y,c = "black")

# plt.show()